In [1]:
import numpy as np

# Replication of Runaway Feedback Loop in Predictive Policing (Ensign et al. 2018)

To replicate the paper of Ensign et al. (2018) it is necessary to describe the model.


### "Protagonists" of the model
- One officer
- Two Neighborhoods (A and B)
- A predictive policing model is implemented

### Dynamics of the model
1. Every day the police officer is sent to one of the neighborhoods
2. The Police officer may or may not observe a crime; if they do observe a crime it is logged and we refer to such log as discovered crime
3. Civils may report crimes, we call this logs reported crimes. 
4. The predicitve policing model is recalibrated using the newest feedback (logs)
5. With the new predictions, the officer is given orders about which neighborhood to go in the next period

### Objective of the model

The goal is to create a predictive model for where to send the officer on each day. Specifically, the goal is to distribute the police officers in proportion to the crime in each area. The ideal result is, if $\eta$ % of the crime ocurrs on A, then the $\eta$ % of police resources is allocate to the same neighborhood. 

### Parameters

- $\lambda_A$ : underlying ground truth crime rate for the neighborhood A
- $d_A$ : rate at which the police visits to neighborhood A result in discovered crimes
- $w_d$ : weight of discovered crimes within all crimes
- $w_r$ : weight of reported crimes within all crimes
- $w_d+w_r=1$
- $r_A$ : rate at which incidents are reported from the neighborhood A
- $C_A$ : total rate of crime data from neighborhood A
- $C_A=w_d\cdot d_A+w_r\cdot r_A$


### Assumptions

1. **PREDICTIVE MODEL**: The officer tosses a coin based on current statistics to decide where to go next.
2. **CONTEXT**: The only information retained about crime is a count.
3. **TRUTH IN CRIME DATA**: if an officer goes to a location A with an underlying ground truth crime rate of $\lambda_A$, the oficer discovers crimes at $\lambda_A$ rate ($\lambda_A=d_A$). 
4. **DISCOVERY ONLY** : Crime data is only collected by an officer's presence in a neighborhood. Neighborhoods with no officers will contribute no crimes to the data

## Construction of the predictive model (Pólya Urn)



## Scenario 1: Non-Uniform crime rates

##### Assumptions

$\lambda_A\neq \lambda_B$